<a href="https://colab.research.google.com/github/JoannaLe/cancer-detection/blob/master/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential, Model

import numpy as np
import os
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# Open Google Cloud Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
import os
# Classifier  
x_path = "gdrive/My Drive/cancer-detection/patch-zoom-4"


print(os.listdir(x_path))
x_arr = []

os.getcwd()
for file in os.listdir(x_path):
  path_file = os.path.join(x_path, file)
  print(path_file)
  content = open(path_file, "r")
  string = content.read()
  arr = np.array(list(string))
  print(arr)
  print(type(arr))
  x_arr.append(arr)
  


['091-patch-0', '091-patch-1', '091-patch-2', '091-patch-3', '091-patch-4', '091-patch-5', '091-patch-6', '091-patch-7', '091-patch-8', '091-patch-9', '091-patch-10', '091-patch-11', '091-patch-12', '091-patch-13', '091-patch-14', '091-patch-15', '091-patch-16', '091-patch-18', '091-patch-17', '091-patch-19', '091-patch-20', '091-patch-21', '091-patch-22', '091-patch-23', '091-patch-24', '091-patch-25', '091-patch-26', '091-patch-27', '091-patch-28', '091-patch-29', '091-patch-30', '091-patch-31', '091-patch-32', '091-patch-33', '091-patch-34', '091-patch-35', '091-patch-36', '091-patch-37', '091-patch-38', '091-patch-39', '091-patch-40', '091-patch-41', '091-patch-42', '091-patch-43', '091-patch-44', '091-patch-45', '091-patch-46', '091-patch-47', '091-patch-48', '091-patch-49', '091-patch-50', '091-patch-51', '091-patch-52', '091-patch-53', '091-patch-54', '091-patch-55', '091-patch-56', '091-patch-57', '091-patch-58', '091-patch-59', '091-patch-60', '091-patch-61', '091-patch-62', '

## Model 1
4 resnet models with a concatenation layer

In [0]:
img_shape = (300,300,3)

model1 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
model1 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
zoom3 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
zoom4 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)

# concatenation layer
concat = Average()([zoom1.output, zoom2.output, zoom3.output, zoom4.output])

full_model = Model([zoom1.input, zoom2.input, zoom3.input, zoom4.input], concat)


In [0]:
# to train, call
EPOCHS = 20
BATCH_SIZE = 32

full_model.fit([X_train1, X_train2, X_train3, X_train4],
               [Y_train1, Y_train2, Y_train3, Y_train4],
                epochs=EPOCHS,
                batch_size=BATCH_SIZE)

score = model.evaluate([X_test1, X_test2, X_test3, X_test4],
                       [Y_test1, Y_test2, Y_test3, Y_test4],
                       batch_size=BATCH_SIZE)

# Model 2
Transfer Learning using weights from Resnet \\
Extract features from Resnet weights and training/test data

In [0]:
img_shape = (300,300,3)

base = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
